In [3]:
import pdfplumber
import pytesseract
import cv2
from PIL import Image

## Paths

In [ ]:
from pathlib import Path

image_path = Path("../testfiles")

#image_file = image_path / "Trek_Allen_test Receipt.jpg"

## Helper Functions

In [9]:
def is_scanned_pdf(pdf_path):
    """ Does not

    :param pdf_path: _description_
    :type pdf_path: _type_
    :return: _description_
    :rtype: _type_
    """
    with pdfplumber.open(pdf_path) as pdf:
        first_page = pdf.pages[0]
        if first_page.extract_text() is None:
            return True
        return False
    
def extract_text_from_pdf(pdf_path):
    """ Imported from original code
    
    Not Used yet

    :param pdf_path: _description_
    :type pdf_path: _type_
    :return: _description_
    :rtype: _type_
    """
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            image = page.to_image()
            page_text = pytesseract.image_to_string(image.original)
            text += page_text
    return text

def preprocess_image(image_path):
    """Preprocess the image for better OCR results.

    :param image_path: _description_
    :type image_path: _type_
    :return: _description_
    :rtype: _type_
    """
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    # Deskewing can be added here with more complex algorithms
    return binary

# Load the image and process 

Preprocess and load into a *large* text area

In [22]:
img = preprocess_image(image_path / 'Walmart_test_receipt.png')

# Perform OCR using pytesseract
text = pytesseract.image_to_string(img)

# Print the size of the extracted text buffer
print(f"Loaded {len(text)} characters")


Loaded 368 characters


### Break the text area into smaller chunks (*sentances*) based on `\n`

Use `re` library

In [23]:
import re

sentences = re.split(r'\n+', text.strip())
print(f"found {len(sentences)} sentences")

found 18 sentences


In [ ]:
for s in sentences:
    print(f"Sentence: {s} | Length: {len(s)}")

## Analyze the extracted text

1. Look for a date and time

In [27]:
import datetime

for s in sentences:
    matches = re.findall(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4}', s)
    if matches:
        print(f"Found date(s) in sentence: {s} | Matches: {matches}")
        try:
            dates = [datetime.datetime.strptime(date, '%m/%d/%y') for date in matches]
            print(f"Parsed dates: {dates}")
        except ValueError as e:
            print(f"Error parsing dates: {e}")

Found date(s) in sentence: 08/10/25 18:15:26 | Matches: ['08/10/25']
Parsed dates: [datetime.datetime(2025, 8, 10, 0, 0)]
